# Assignment 4
## Description
In this assignment you must read in a file of metropolitan regions and associated sports teams from [assets/wikipedia_data.html](assets/wikipedia_data.html) and answer some questions about each metropolitan region. Each of these regions may have one or more teams from the "Big 4": NFL (football, in [assets/nfl.csv](assets/nfl.csv)), MLB (baseball, in [assets/mlb.csv](assets/mlb.csv)), NBA (basketball, in [assets/nba.csv](assets/nba.csv) or NHL (hockey, in [assets/nhl.csv](assets/nhl.csv)). Please keep in mind that all questions are from the perspective of the metropolitan region, and that this file is the "source of authority" for the location of a given sports team. Thus teams which are commonly known by a different area (e.g. "Oakland Raiders") need to be mapped into the metropolitan region given (e.g. San Francisco Bay Area). This will require some human data understanding outside of the data you've been given (e.g. you will have to hand-code some names, and might need to google to find out where teams are)!

For each sport I would like you to answer the question: **what is the win/loss ratio's correlation with the population of the city it is in?** Win/Loss ratio refers to the number of wins over the number of wins plus the number of losses. Remember that to calculate the correlation with [`pearsonr`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.pearsonr.html), so you are going to send in two ordered lists of values, the populations from the wikipedia_data.html file and the win/loss ratio for a given sport in the same order. Average the win/loss ratios for those cities which have multiple teams of a single sport. Each sport is worth an equal amount in this assignment (20%\*4=80%) of the grade for this assignment. You should only use data **from year 2018** for your analysis -- this is important!

## Notes

1. Do not include data about the MLS or CFL in any of the work you are doing, we're only interested in the Big 4 in this assignment.
2. I highly suggest that you first tackle the four correlation questions in order, as they are all similar and worth the majority of grades for this assignment. This is by design!
3. It's fair game to talk with peers about high level strategy as well as the relationship between metropolitan areas and sports teams. However, do not post code solving aspects of the assignment (including such as dictionaries mapping areas to teams, or regexes which will clean up names).
4. There may be more teams than the assert statements test, remember to collapse multiple teams in one city into a single value!

## Question 1
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NHL** using **2018** data.

In [57]:
%%capture
#RUN FIRST, installs a missing library
import sys
!{sys.executable} -m pip install lxml==4.4.1

In [2]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

nhl_df = pd.read_csv("assets/nhl.csv")

## read_html devuele list of DF. El [1] toma el dataframe de Metropolitan Area
cities = pd.read_html("assets/wikipedia_data.html")[1]
cities = cities.iloc[:-1,[0,3,5,6,7,8]]

for x in cities: ## x == nombre de las columnas, igual a for x in cities.columns:
    cities[x] = cities[x].str.replace("\[.+\]|—", "", regex=True)
    #cities[x].replace(to_replace="", value=np.nan, inplace=True)
    #el np.nan es numero float y ya no puede hacer el str.contains en la siguiente parte
    
    #x_regex = re.sub("[\(\[].+[\)\]]", "", x)
    #cities.rename(columns={x : x_regex.strip()}, inplace=True)
    cities.rename(columns={x : re.sub("[\(\[].+[\)\]]", "", x).strip()}, inplace=True)

nhl_df["team"] = nhl_df["team"].str.replace("[*]", "", regex=True)

#print(nhl_df.loc[0:10, ["team", "W", "L", "year"]])
#cities.iloc[0:20]

In [9]:
## Formas de hacer busqueda
nhl_df[nhl_df.eq("Philadelphia Flyers").any(1)]
## .eq() busca valor "xxx" en el df (tiene que ser exacto)
## str.contains es busqueda aproximada de la palabra (no tiene que ser exacto)


## separar los nombres de la tabla HTML y asignar el valor de cada Metropolitan Area en un dict()
team_names = {}
i = 0
for names in cities["NHL"]:
    for name in names.split():
        if len(name) < 1: continue
        team_names[name] = cities["Metropolitan area"].iloc[i]
        #print(name)
    i = i + 1
print("Dict: {} \nTotal: {}".format(team_names, len(team_names)))


## str.contains() busca cualquier parecido // .eq() tiene que ser exacto // find()
nhl_df["area"] = ""
for x in team_names:
    #nhl_df["area"].loc[nhl_df["team"].eq(x).any()] = team_names[x] #este no funciona
    
    nhl_df["area"].loc[nhl_df["team"].str.contains(r"\b{}\b".format(x), regex=True)] = team_names[x]
    ## este si funciona pero requirio de word boundary \bxx\b de regex para hacer el match con valores exactos
    ## porque habia St. Louis Blues y Columbus Blue Jackets que los ponia bajo una misma region
    
    
nhl_df[["team", "area"]].where(nhl_df["area"] == "Los Angeles").dropna() ## mismo resultado
nhl_df[["team", "area"]][nhl_df["area"] == "Los Angeles"] ## mismo resultado
#print(nhl_df.loc[10:21, ["team", "area", "W", "L", "year"]])

#comprobacion de que team_names asigno bien
print(nhl_df[["team", "area"]][nhl_df["area"]=="Columbus"])
print(nhl_df["team"].loc[nhl_df["team"].str.contains("Blue")])

Dict: {'Rangers': 'New York City', 'Islanders': 'New York City', 'Devils': 'New York City', 'Kings': 'Los Angeles', 'Ducks': 'Los Angeles', 'Sharks': 'San Francisco Bay Area', 'Blackhawks': 'Chicago', 'Stars': 'Dallas–Fort Worth', 'Capitals': 'Washington, D.C.', 'Flyers': 'Philadelphia', 'Bruins': 'Boston', 'Wild': 'Minneapolis–Saint Paul', 'Avalanche': 'Denver', 'Panthers': 'Miami–Fort Lauderdale', 'Coyotes': 'Phoenix', 'Red': 'Detroit', 'Wings': 'Detroit', 'Maple': 'Toronto', 'Leafs': 'Toronto', 'Lightning': 'Tampa Bay Area', 'Penguins': 'Pittsburgh', 'Blues': 'St. Louis', 'Predators': 'Nashville', 'Sabres': 'Buffalo', 'Canadiens': 'Montreal', 'Canucks': 'Vancouver', 'Blue': 'Columbus', 'Jackets': 'Columbus', 'Flames': 'Calgary', 'Senators': 'Ottawa', 'Oilers': 'Edmonton', 'Jets': 'Winnipeg', 'Golden': 'Las Vegas', 'Knights': 'Las Vegas', 'Hurricanes': 'Raleigh'} 
Total: 35
                      team      area
13   Columbus Blue Jackets  Columbus
47   Columbus Blue Jackets  Columbus


In [11]:
## merge previo al groupby para tener un solo dato por Metropolitan area 
nhl_merge = pd.merge(nhl_df[nhl_df["year"]==2018][["area", "team", "W", "L", "year"]],
                     cities[["Metropolitan area", "Population"]],
                     left_on="area", right_on="Metropolitan area")

nhl_merge = nhl_merge.astype({"W": int, "L": int, "Population": int})
nhl_merge["W/L Ratio"] = nhl_merge["W"] / (nhl_merge["W"] + nhl_merge["L"])
#print(nhl_merge.head())


print(nhl_merge[nhl_merge["team"].str.contains("Blue")])
print(nhl_merge[nhl_merge["area"]=="New York City"])

grouped = nhl_merge.groupby(by="Metropolitan area").agg({"W/L Ratio": np.mean, "Population": np.mean})

print("Total groupby por area {}".format(grouped.shape)) ## Resultado 28 (tiene que ser 28)
#print(grouped.sort_values(by="Metropolitan area"))

"""
for group, frame in nhl_merge.groupby("area"):
    total_wins = np.sum(pd.to_numeric(frame['W']))
    total_losses = np.sum(pd.to_numeric(frame['L']))
    total_matches = total_wins + total_losses
    ratio = (total_wins / total_matches)
    out_dict = {
        'Area': group,
        'Ratio': ratio}
    #print(out_dict)
nhl_merge.iloc[10:20]
"""
grouped

         area                   team   W   L  year Metropolitan area  \
11   Columbus  Columbus Blue Jackets  45  30  2018          Columbus   
20  St. Louis        St. Louis Blues  44  32  2018         St. Louis   

    Population  W/L Ratio  
11     2041520   0.600000  
20     2807002   0.578947  
             area                team   W   L  year Metropolitan area  \
12  New York City   New Jersey Devils  44  29  2018     New York City   
13  New York City  New York Islanders  35  37  2018     New York City   
14  New York City    New York Rangers  34  39  2018     New York City   

    Population  W/L Ratio  
12    20153634   0.602740  
13    20153634   0.486111  
14    20153634   0.465753  
Total groupby por area (28, 2)


,W/L Ratio,Population
Metropolitan area,,
Boston,0.714286,4794447.0
Buffalo,0.357143,1132804.0
Calgary,0.513889,1392609.0
Chicago,0.458333,9512999.0
Columbus,0.600000,2041520.0
Dallas–Fort Worth,0.567568,7233323.0
Denver,0.589041,2853077.0
Detroit,0.434783,4297617.0
Edmonton,0.473684,1321426.0


In [2]:
def nhl_correlation(): 
    import pandas as pd
    import numpy as np
    import scipy.stats as stats
    import re
    
    nhl_df = pd.read_csv("assets/nhl.csv")
    cities = pd.read_html("assets/wikipedia_data.html")[1]
    cities = cities.iloc[:-1,[0,3,5,6,7,8]]
    for x in cities:
        cities[x] = cities[x].str.replace("\[.+\]|—", "", regex=True)
        cities.rename(columns={x : re.sub("[\(\[].+[\)\]]", "", x).strip()}, inplace=True)
    nhl_df["team"] = nhl_df["team"].str.replace("[*]", "", regex=True)
    
    team_names = {}
    i = 0
    for names in cities["NHL"]:
        for name in names.split():
            if len(name) < 1: continue
            team_names[name] = cities["Metropolitan area"].iloc[i]
        i = i + 1
        
    nhl_df["area"] = ""
    for x in team_names:
        nhl_df["area"].loc[nhl_df["team"].str.contains(r"\b{}\b".format(x), regex=True)] = team_names[x]
    
    nhl_merge = pd.merge(nhl_df[nhl_df["year"]==2018][["area", "team", "W", "L", "year"]],
                         cities[["Metropolitan area", "Population"]],
                         left_on="area", right_on="Metropolitan area")
    nhl_merge = nhl_merge.astype({"W": int, "L": int, "Population": int})
    nhl_merge["W/L Ratio"] = nhl_merge["W"] / (nhl_merge["W"] + nhl_merge["L"])
    
    grouped = nhl_merge.groupby(by="area").agg({"W/L Ratio": np.mean, "Population": np.mean})
    
    #raise NotImplementedError()
    
    population_by_region = grouped["Population"] # pass in metropolitan area population from cities
    win_loss_by_region = grouped["W/L Ratio"] # pass in win/loss ratio from nhl_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q1: Your lists must be the same length"
    assert len(population_by_region) == 28, "Q1: There should be 28 teams being analysed for NHL"
    
    corr, pval = stats.pearsonr(population_by_region, win_loss_by_region)
    return corr

nhl_correlation()

0.012486162921209907

## Question 2
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NBA** using **2018** data.

In [3]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

nba_df=pd.read_csv("assets/nba.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]


def nba_correlation():
    for x in cities:
        cities[x] = cities[x].str.replace("\[.+\]|—", "", regex=True)
        cities.rename(columns={x : re.sub("[\(\[].+[\)\]]", "", x).strip()}, inplace=True)

    nba_df["team"] = nba_df["team"].str.replace("[*\(].+", "", regex=True).str.strip()
    #print(nba_df["team"].iloc[25:40])
    
    team_names = {}
    i = 0
    for names in cities["NBA"]:
        for name in names.split():
            if len(name) < 1: continue
            team_names[name] = cities["Metropolitan area"].iloc[i]
        i = i + 1
    #print("Dict \n{}".format(team_names))

    nba_df["area"] = ""
    for x in team_names:
        nba_df["area"].loc[nba_df["team"].str.contains(r"\b{}\b".format(x), regex=True)] = team_names[x]

    #nba_df[["area", "team", "year"]].value_counts()

##comprobacion de que team_names asigno bien
    #print(nba_df[["team", "area"]][nba_df["area"]=="New York City"])
    #print(nba_df["team"].loc[nba_df["team"].str.contains("Net")])
    #print(nba_df.groupby(by="area")["team"].count())


    nba_merge = pd.merge(nba_df[nba_df["year"]==2018][["area", "team", "W", "L", "W/L%", "year"]],
                        cities[["Metropolitan area", "Population"]],
                        left_on="area", right_on="Metropolitan area")
    nba_merge = nba_merge.astype({"W": int, "L": int, "W/L%": float, "Population": int})
    #print(nba_merge.head())
    
    grouped = nba_merge.groupby(by="area").agg({"W/L%": np.mean, "Population": np.mean})
    #print(grouped)
    
    population_by_region = grouped["Population"] # pass in metropolitan area population from cities
    win_loss_by_region = grouped["W/L%"] # pass in win/loss ratio from nba_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q2: Your lists must be the same length"
    assert len(population_by_region) == 28, "Q2: There should be 28 teams being analysed for NBA"
    
    corr, pval = stats.pearsonr(population_by_region, win_loss_by_region)
    return corr

nba_correlation()

-0.17636350642182935

## Question 3
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **MLB** using **2018** data.

In [7]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

mlb_df=pd.read_csv("assets/mlb.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

def mlb_correlation():
    for x in cities:
        cities[x] = cities[x].str.replace("\[.+\]|—", "", regex=True)
        cities.rename(columns={x : re.sub("[\(\[].+[\)\]]", "", x).strip()}, inplace=True)

    mlb_df["team"] = mlb_df["team"].str.strip()
    #print(mlb_df["team"].iloc[25:40])
    
    team_names = {}
    i = 0
    for names in cities["MLB"]:
        for name in names.split():
            if len(name) < 1: continue
            team_names[name] = cities["Metropolitan area"].iloc[i]
        i = i + 1
    #print("Dict \n{}".format(team_names))
    
    mlb_df["area"] = ""
    for x in team_names:
        if x == "Sox": continue ##hay White Sox y Red Sox en diferentes ciudades
        mlb_df["area"].loc[mlb_df["team"].str.contains(r"\b{}\b".format(x), regex=True)] = team_names[x]


##comprobacion de que team_names asigno bien
    #print(mlb_df[["team", "area"]][mlb_df["area"]=="New York City"])
    #print(mlb_df[["team", "area"]].loc[mlb_df["team"].str.contains("Sox")])

    mlb_merge = pd.merge(mlb_df[mlb_df["year"]==2018][["area", "team", "W", "L", "W-L%", "year"]],
                        cities[["Metropolitan area", "Population"]],
                        left_on="area", right_on="Metropolitan area")
    mlb_merge = mlb_merge.astype({"W": int, "L": int, "W-L%": float, "Population": int})
    #print(mlb_merge.head())
    
    grouped = mlb_merge.groupby(by="area").agg({"W-L%": np.mean, "Population": np.mean})
    #print(grouped)
   
    #raise NotImplementedError()
    population_by_region = grouped["Population"] # pass in metropolitan area population from cities
    win_loss_by_region = grouped["W-L%"] # pass in win/loss ratio from mlb_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q3: Your lists must be the same length"
    assert len(population_by_region) == 26, "Q3: There should be 26 teams being analysed for MLB"

    corr, pval = stats.pearsonr(population_by_region, win_loss_by_region)
    return corr

mlb_correlation()

0.15003737475409498

## Question 4
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NFL** using **2018** data.

In [42]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

nfl_df=pd.read_csv("assets/nfl.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]



def nfl_correlation(): 
    for x in cities:
        cities[x] = cities[x].str.replace("\[.+\]|—", "", regex=True)
        cities.rename(columns={x : re.sub("[\(\[].+[\)\]]", "", x).strip()}, inplace=True)

    nfl_df["team"] = nfl_df["team"].str.replace("[*+]", "", regex=True).str.strip()
    #print(nfl_df["team"].loc[38:54])
    
    team_names = {}
    i = 0
    for names in cities["NFL"]:
        for name in names.split():
            if len(name) < 1: continue
            team_names[name] = cities["Metropolitan area"].iloc[i]
        i = i + 1
    #print("Dict \n{}".format(team_names))
    #print(team_names.values() == "Las Vegas")
    ## en wikipedia actualizado San Francisco tiene un equipo y Raiders es en Las Vegas (post2019),
    ## en version del ejercicio San Francisco tiene dos equipos (49ers y Oakland Raiders)
    
    nfl_df["area"] = ""
    for x in team_names:
        nfl_df["area"].loc[nfl_df["team"].str.contains(r"\b{}\b".format(x), regex=True)] = team_names[x]


##comprobacion de que team_names asigno bien
    #print(nfl_df[["team", "area"]][nfl_df["area"]=="San Francisco Bay Area"])
    #print(nfl_df[["team", "area"]].loc[nfl_df["team"].str.contains("49ers")])


    nfl_merge = pd.merge(nfl_df[nfl_df["year"]==2018][["area", "team", "W", "L", "W-L%", "year"]],
                        cities[["Metropolitan area", "Population"]],
                        left_on="area", right_on="Metropolitan area")
    nfl_merge = nfl_merge.astype({"W": int, "L": int, "W-L%": float, "Population": int})
    #print(nfl_merge.head())
    
    grouped = nfl_merge.groupby(by="area").agg({"W-L%": np.mean, "Population": np.mean})
    #print(grouped)
   
    #raise NotImplementedError()
    population_by_region = grouped["Population"] # pass in metropolitan area population from cities
    win_loss_by_region = grouped["W-L%"] # pass in win/loss ratio from nfl_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q4: Your lists must be the same length"
    assert len(population_by_region) == 29, "Q4: There should be 29 teams being analysed for NFL"

    corr, pval = stats.pearsonr(population_by_region, win_loss_by_region)
    return corr

nfl_correlation()

0.004282141436393024

## Question 5
In this question I would like you to explore the hypothesis that **given that an area has two sports teams in different sports, those teams will perform the same within their respective sports**. How I would like to see this explored is with a series of paired t-tests (so use [`ttest_rel`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.ttest_rel.html)) between all pairs of sports. Are there any sports where we can reject the null hypothesis? Again, average values where a sport has multiple teams in one region. Remember, you will only be including, for each sport, cities which have teams engaged in that sport, drop others as appropriate. This question is worth 20% of the grade for this assignment.

In [137]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

mlb_df=pd.read_csv("assets/mlb.csv")
nhl_df=pd.read_csv("assets/nhl.csv")
nba_df=pd.read_csv("assets/nba.csv")
nfl_df=pd.read_csv("assets/nfl.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]


def sports_team_performance():
    for x in cities:
        cities[x] = cities[x].str.replace("\[.+\]|—", "", regex=True)
        cities.rename(columns={x : re.sub("[\(\[].+[\)\]]", "", x).strip()}, inplace=True)

    nhl_df["team"] = nhl_df["team"].str.replace("[*]", "", regex=True)
    nba_df["team"] = nba_df["team"].str.replace("[*\(].+", "", regex=True).str.strip()
    mlb_df["team"] = mlb_df["team"].str.strip()
    nfl_df["team"] = nfl_df["team"].str.replace("[*+]", "", regex=True).str.strip()
    
    ## intento de hacer un solo metodo para obtener todos los nombres de los equipos
    Big4 = {"NHL": nhl_df, "NBA": nba_df, "MLB": mlb_df, "NFL": nfl_df}
    team_names = {}
    team_names_list = []
    for sport in Big4: #itera solo en las keys()
        i = 0
        for names in cities[sport]:
            if i > len(cities[sport]): break
            for name in names.split():
                if len(name) <= 1: continue
                city_region = cities["Metropolitan area"].iloc[i]
                #team_names[name] = cities["Metropolitan area"].iloc[i] #dict_keys no son unique/Ej Rangers en NHl y MBL
                team_names_list.append((name, city_region))
            i += 1
        Big4[sport]["area"] = "" ## Big4[sport]["area"] pasa a ser xxx_df["area"]
        for x in team_names_list:
            if x[0] == "Sox": continue
            regex_search = r"\b{}\b".format(x[0])
            Big4[sport]["area"].loc[Big4[sport]["team"].str.contains(regex_search, regex=True)] = x[1]
    
    ## Comprobaciones
    #print(nhl_df[["team", "area"]][nhl_df["area"]=="Las Vegas"])
    #print(nhl_df[["team", "area"]][nhl_df["team"].str.contains("Rangers")])
    #print(mlb_df[["team", "area"]][mlb_df["team"].str.contains("Rangers")])
    #print(nfl_df[["team", "area"]][nfl_df["area"]=="San Francisco Bay Area"])
    #print(nfl_df[["team", "area"]].loc[nfl_df["team"].str.contains("49ers")])
    #print(mlb_df[["team", "area"]][mlb_df["area"]=="New York City"])
    #print(mlb_df[["team", "area"]].loc[mlb_df["team"].str.contains("Sox")])


    def group_df(x):
        df_merge = pd.merge(Big4[x][Big4[x]["year"]==2018][["area", "team", "W", "L", "year", "League"]],
                        cities["Metropolitan area"],
                        left_on="area", right_on="Metropolitan area")
        #df_merge = df_merge.astype({"W": int, "L": int, "Population": int})
        df_merge["W/L Ratio"] = df_merge["W"].astype(int) / (df_merge["W"].astype(int) + df_merge["L"].astype(int))
        grouped = df_merge.groupby("area").agg(Ratio=("W/L Ratio", np.mean))
        # rename colum a Ratio // .agg(nombre=("columna", func))
        return grouped
    
    # Note: p_values is a full dataframe, so df.loc["NFL","NBA"] should be the same as df.loc["NBA","NFL"] and
    # df.loc["NFL","NFL"] should return np.nan
    ## // OBS al hacer stats.ttest_rel con grupos iguales resultado es NaN
    sports = ['NFL', 'NBA', 'NHL', 'MLB']
    p_values = pd.DataFrame({k:np.nan for k in sports}, index=sports)
    
    for x in Big4:
        #print(len(group_df(x))) # NHL 28 / NBA 28 / MLB 26 / NFL 29 ##comprobacion de group_df(x)
        for y in Big4:
            merge_df = pd.merge(group_df(x), group_df(y), how="inner", on="area")
            #print(merge.head(3))
            #print("For {} & {}".format(x, y))
            t_test, pval = stats.ttest_rel(merge_df["Ratio_x"], merge_df["Ratio_y"])
            p_values.loc[x, y] = pval

    #raise NotImplementedError()
    assert abs(p_values.loc["NBA", "NHL"] - 0.02) <= 1e-2, "The NBA-NHL p-value should be around 0.02"
    assert abs(p_values.loc["MLB", "NFL"] - 0.80) <= 1e-2, "The MLB-NFL p-value should be around 0.80"
    return p_values


sports_team_performance()

,NFL,NBA,NHL,MLB
NFL,NaN,0.941792,0.030883,0.802069
NBA,0.941792,NaN,0.022297,0.950540
NHL,0.030883,0.022297,NaN,0.000708
MLB,0.802069,0.950540,0.000708,NaN
